# Activity 5: Load data - Azure SQL Database

#### <i>The Azure SQL Workshop - Module 2</i>

<p style="border-bottom: 1px solid lightgrey;"></p>

In this activity, you'll get to see how you can bulk load data into Azure SQL Database.  


**Set up - Attach the notebook to Azure SQL Database**   

0. You should have opened this file using Azure Data Studio. If you didn't, please refer to Module 2 Activity 3 in the main Module 2 file to get set up.  
1. In the bar at the top of this screen, confirm or change the "Kernel" to **SQL**. This determines what language the code blocks in the file are. In this case, that language is SQL.  
2. For "Attach to", use the drop-down to select **Change Connection**. From the Recent Connections pane, you should be able to select your Azure SQL Database logical server and be sure to **select your AdventureWorks database as the database to connect to**.  

Now that you're set up, you should read the text cells and "Run" the code cells by selecting the play button that appears in the left of a code cell when you hover over it.  
> Some of the cells have been run before, this is just to show you the expected result from the testing of the labs. If you choose not to complete the labs/prerequisites, do not run any cells, just review the results.      


When you're bulk loading data, it has to come from somewhere. In Azure, it's very common to store or dump data into an [Azure Blob Storage](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-blobs-introduction) because Blob storage is optimized for storing massive amounts of unstructured data at a relatively low cost.   

In this scenario, AdventureWorks is receiving store return data based on store identification number (e.g. 1, 2, etc.) This return data is being stored in `.dat` files which are then pushed into Azure Blob storage.  

Within blob storage, there exists three types of resources:  
* Storage account: this provides a unique namespace for a storage account, and a way to connect or access it  
* Containers: these are used to organize a set of blobs. A storage account can have an unlimited number of containers  
* Blobs: there are several types of blobs but we will use Block blobs that can store text and binary data that can be managed individually.  

Now, once the data is in blob storage, Azure SQL needs a way to access it. You can do that by [creating an external data source](https://docs.microsoft.com/en-us/sql/t-sql/statements/create-external-data-source-transact-sql?view=azuresqldb-current) that has access to the Azure Storage account.  

You can [control access to Azure Storage accounts](https://docs.microsoft.com/en-us/azure/storage/common/storage-account-overview?toc=%2fazure%2fstorage%2fblobs%2ftoc.json#control-access-to-account-data) through Azure Active Directory, Shared Key authorization, or with a Shared access signature (SAS). The link points to more details, but we will use SAS for this exercise.  

If you want to read more about how SAS works with regards to Azure Storage, please [read here](https://docs.microsoft.com/en-us/azure/storage/common/storage-sas-overview) before continuing.  



**Step 1 - Create a table and schema**  

First, we need to create a table and schema for our data to be loaded into. This is pretty straightforward, good old-fashioned T-SQL.

In [6]:
IF SCHEMA_ID('DataLoad') IS NULL 
EXEC ('CREATE SCHEMA DataLoad')

CREATE TABLE DataLoad.store_returns
(
    sr_returned_date_sk             bigint,
    sr_return_time_sk               bigint,
    sr_item_sk                      bigint           ,
    sr_customer_sk                  bigint,
    sr_cdemo_sk                     bigint,
    sr_hdemo_sk                     bigint,
    sr_addr_sk                      bigint,
    sr_store_sk                     bigint,
    sr_reason_sk                    bigint,
    sr_ticket_number                bigint           ,
    sr_return_quantity              integer,
    sr_return_amt                   float,
    sr_return_tax                   float,
    sr_return_amt_inc_tax           float,
    sr_fee                          float,
    sr_return_ship_cost             float,
    sr_refunded_cash                float,
    sr_reversed_charge              float,
    sr_store_credit                 float,
    sr_net_loss                     float

) 

Commands completed successfully.

Total execution time: 00:00:00.016

**Step 2 - Create a `MASTER KEY`**  

Leveraging [an example in the docs](https://docs.microsoft.com/en-us/sql/t-sql/statements/bulk-insert-transact-sql?view=sql-server-ver15#f-importing-data-from-a-file-in-azure-blob-storage) you learn that a `MASTER KEY` is required to create a `DATABASE SCOPED CREDENTIAL` since the blob storage is not configured to allow public (anonymous) access.  

So, let's first create a `MASTER KEY`

In [7]:
CREATE MASTER KEY 
ENCRYPTION BY PASSWORD='MyComplexPassword00!';

Commands completed successfully.

Total execution time: 00:00:00.015

**Step 3 - Create a `DATABASE SCOPED CREDENTIAL`**  

A `MASTER KEY` is required to create a `DATABASE SCOPED CREDENTIAL`, which we can now create. The credential refers to the Azure blob storage account and the `data/` portion specifies the container where the store return data is located.  

We use `SHARED ACCESS SIGNATURE` as the identity which SQL knows how to interpret, and the secret provided is the SAS token that you can generate from the Azure blob storage account.  

> Note: the `?` at the beginning of the SAS token should be removed  

> Note: be sure to update the SECRET below


In [8]:
CREATE DATABASE SCOPED CREDENTIAL [https://azuresqlworkshopsa.blob.core.windows.net/data/]
WITH IDENTITY = 'SHARED ACCESS SIGNATURE',
SECRET = 'Go to URL aka.ms/aswsas20 and copy';

Commands completed successfully.

Total execution time: 00:00:00.011

**Step 4 - Create an external data source to the container**  

> Note: `LOCATION` doesn't have a trailing `/`, even through the `CREDENTIAL` does.

In [9]:
CREATE EXTERNAL DATA SOURCE dataset
WITH 
(
    TYPE = BLOB_STORAGE,
    LOCATION = 'https://azuresqlworkshopsa.blob.core.windows.net/data',
    CREDENTIAL = [https://azuresqlworkshopsa.blob.core.windows.net/data/]
);

Commands completed successfully.

Total execution time: 00:00:00.018

**Step 5 - `BULK INSERT` a single file**   

You're finally ready to `BULK INSERT` one of the store return files. 

Run the following cell, and while it completes, review the comments.

In [10]:
SET NOCOUNT ON -- Reduce network traffic by stopping the message that shows the number of rows affected
 BULK INSERT DataLoad.store_returns -- Table you created in Step 1
 FROM 'dataset/store_returns/store_returns_1.dat' -- Within the container, the location of the file
     WITH (
			DATA_SOURCE = 'dataset' -- Using the External data source from Step 4
			,DATAFILETYPE = 'char' 
	        ,FIELDTERMINATOR = '\|' 
	        ,ROWTERMINATOR = '\|\n' 
            ,BATCHSIZE=100000 -- Reduce network traffic by inserting in batches
            , TABLOCK -- Minimize number of log records for the insert operation
           )

Commands completed successfully.

Total execution time: 00:01:51.021

For now, let's check how many rows were inserted into our table:

In [11]:
select count(*) from DataLoad.store_returns

Commands completed successfully.

Total execution time: 00:00:00.820

(No column name)
2807797


If you want to run throught the exercise again, run the following code to reset what you've done.

In [ ]:
DROP EXTERNAL DATA SOURCE dataset
DROP DATABASE SCOPED CREDENTIAL [https://azuresqlworkshopsa.blob.core.windows.net/data/]
DROP TABLE DataLoad.store_returns
DROP MASTER KEY

You've seen how you can optimize bulk inserts in this activity. We are really just skimming the surface here, and in the Performance module you'll learn about other ways you can optimize performance. In future development of this workshop, we plan to add more here (things like parallelism are not mentioned but can help). Until then, You can check the [documentation (under **How to** > **Load and move data**)](https://docs.microsoft.com/en-us/azure/sql-database/sql-database-load-from-csv-with-bcp) for more information.  